채팅과 같음 장식으로 챗봇과 여러 쿼리를 통한 질문과 응답을 위해서는 대화형 메모리가 필요하다. 메모리가 있어야 일관된 대화를 할 수 있다. 메모리가 없다면 매번 수행하는 질문은 기존에 했던 대화와는 아무런 관계가 없는 독립적인 인풋으로 간주가 될 것이다.
![메모리 유무에 따른 LLM의 반응](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F927ca8cc5d92ee75f36d7eb4bef4685c4e3118e5-2880x1370.png&w=3840&q=75)

그림에서 하늘색은 사용자의 프롬프트이고 회색은 LLM의 대답이다. 대화형 메모리가 없는 오른쪽의 경우 LLM은 이전의 상호작용(interaction)에 대해서 모르기 때문에 대답을 하지 못하는 것을 볼 수 있다.

메모리가 있어야 이전의 상호작용을 기억을 할 수 있다. 기본적으로 LLM은 무상태이다(stateless). 무상태라는 것은 매번 들어오는 쿼리가 다른 상호작용과는 상관없이 독립적으로 처리가 된다는 것이다. 즉, 무상태 에이전트가 알 수 있는 것은 현재의 인풋 뿐이다.

챗봇을 포함한 많은 애플리케이션은 이전의 상호작용을 기억을 하는 것이 필요하다. 대화형 메모리를 통해서 이것이 가능하다.

대화형 메모리를 구현하는 방법은 여러가지가 있다. Langchain에서는 모든 대화형 메모리는 <b>ConversationChain</b>를 기반으로 해서 구현되어 있다.

## ConversationChain
코드를 통해 바로 알아보자.

In [2]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI()

conversation = ConversationChain(llm=llm)

print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


위 결과와 같이 프롬프트에는 인간(사용자)와 AI 사이의 대화가 들어 있다고 말을 하고 있다. 또한 환상(hallucinations)을 없애기 위해(줄이기 위해) 모르면 모른다고 말을 해야 한다고 제시를 하고 있다. 물론 이렇게 말을 한다고 해서 hallucination 문제를 없앨 수는 없고 약간 도움을 준다. 환각을 없애기 위한 다른 방법은 다음 장에서 다룬다.

지금 여기서 중요한 부분은 {history} 파라미터이다. {input}은 사용자의 가장 최근의 쿼리이다. 그림으로 표현하면 아래와 같다.
![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F368f6f63b4d8d5985cbf2d3fb7407eaf01608e84-1750x1080.png&w=1920&q=75)

이미 눈치를 챘겠지만, LLM 자체는 stateless이다. 단지 대화형 메모리를 어딘가에다가 만들어 놓고, 매 쿼리마다 그 대화를 모아다가 쿼리와 함께 {history} 부분에 채워 넣어서 LLM에게 던지는 것이다.

## 대화형 메모리의 형태(Forms of Conversational Memory)

ConversationalChain을 바탕으로 해서 우리는 여러가지 형태의 대화형 메모리를 사용할 수가 있다. 여러가지 형태라 함은 {history} 부분에 들어갈 텍스트를 변화를 시킬 수 있다는 뜻이다.

### ConversationBufferMemory

ConversationBufferMemory는 Langchain에서 가장 간단한(straightforward) 대화형 메모리이다. 앞에서 봤듯이 과거의 대화를 그냥 그 상태 대로(raw) {history} 부분에 끼워넣는 역할을 한다.

In [3]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buff = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation_buff("Good morning AI!")

/Users/minsoojo/Library/Caches/pypoetry/virtualenvs/pinecone-ai-handbook-m0t52s90-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'input': 'Good morning AI!',
 'history': '',
 'response': ' Good morning Human! How are you today? Is there anything I can assist you with?'}

대화형 에이전트로부터 첫번재 답을 얻었다. 기존 대화를 기억하고 있어야만 의미있는 대답을 할 수 있도록 프롬프트를 만들어 가면서 어떻게 진행이 되는 지를 알아보자. 그리고 기존 대화를 포함하면서 토큰이 얼마나 사용되는 지를 알게 되면 어떻게 동작하는 지 더 잘 알 수 있기 때문에 예전처럼 콜백 함수를 하나 만들어서 사용하자.

In [4]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'총 소요 토큰: {cb.total_tokens}')
    return result

count_tokens(conversation_buff, "My interest here is to explore the potential of integrating Large Language Models with external knowledge")

/Users/minsoojo/Library/Caches/pypoetry/virtualenvs/pinecone-ai-handbook-m0t52s90-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


총 소요 토큰: 137


" That's a fascinating topic, Human! I have some knowledge on this subject, as I am a large language model myself. What specific aspects of integrating external knowledge are you interested in?"

In [5]:
count_tokens(
    conversation_buff,
    "I just want to analyze the different possibilities. What can you thif of?"
)

총 소요 토큰: 258


' Well, there are a few different approaches to integrating external knowledge with large language models. One method is to use pre-trained knowledge graphs, which contain structured information about different entities and their relationships. Another approach is to use text corpora, which contain unstructured information and can be used to train the language model on specific domains or topics. Additionally, some models use both structured and unstructured data to enhance their understanding and generate more accurate responses. Is there anything else you would like me to elaborate on?'

In [6]:
count_tokens(
    conversation_buff,
    "Which data source types could be used to give context to the model?"
)

총 소요 토큰: 354


" There are several data source types that can be used to give context to a large language model. Some common types include datasets of text such as news articles, books, or web pages. Other sources could include structured data such as databases, knowledge graphs, or even images and videos. It really depends on the specific use case and the type of information needed to enhance the model's understanding."

In [7]:
count_tokens(
    conversation_buff,
    "What is my aim again?"
)

총 소요 토큰: 414


' Your aim is to explore the potential of integrating large language models with external knowledge. This could involve analyzing different approaches, data sources, and potential benefits of integrating external knowledge with language models. Is there anything else I can assist you with, Human?'

위의 일련의 출력을 보면 사용되는 토큰양은 점점 더 증가하며, 기존의 질문을 잘 기억하고 있는 것을 볼 수 있다.
LangSmith[^1]를 통해 trace를 해 보면 맨 아래의 체인을 위한 인풋은 아래와 같았다.

> {
 "input": "What is my aim again?",
 "history": 
    "
    Human: Good morning AI!\n
    AI:  Good morning Human! How are you today? Is there anything I can assist you with?\n
    Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge\n
    AI:  That's a fascinating topic, Human! I have some knowledge on this subject, as I am a large language model myself. What specific aspects of integrating external knowledge are you interested in?\n
    Human: I just want to analyze the different possibilities. What can you thif of?\n
    AI:  Well, there are a few different approaches to integrating external knowledge with large language models. One method is to use pre-trained knowledge graphs, which contain structured information about different entities and their relationships. Another approach is to use text corpora, which contain unstructured information and can be used to train the language model on specific domains or topics. Additionally, some models use both structured and unstructured data to enhance their understanding and generate more accurate responses. Is there anything else you would like me to elaborate on?\n
    Human: Which data source types could be used to give context to the model?\n
    AI:  There are several data source types that can be used to give context to a large language model. Some common types include datasets of text such as news articles, books, or web pages. Other sources could include structured data such as databases, knowledge graphs, or even images and videos. It really depends on the specific use case and the type of information needed to enhance the model's understanding."
}

[^1] LangSmith는 Langchain 플랫폼에서 제공하는 trace 툴이다. 이 툴을 이용하면 우리의 애플리케이션이 LLM과 어떻게 상호작용을 하는 지, 토큰은 얼마나 사용되는 지 등을 상세하게 알 수 있다. 실행할 때 환경변수에 LANGCHAIN_TRACING_V2 와 LANGCHAIN_API_KEY 두 개만 정확하게 설정해 놓으면 https://smith.langchain.com 에서 살펴볼 수 있다.

LangSmith를 사용하지 않고도 아래와 같이 메모리를 확인해 볼 수 있다.

In [8]:
print(conversation_buff.memory.buffer)

Human: Good morning AI!
AI:  Good morning Human! How are you today? Is there anything I can assist you with?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI:  That's a fascinating topic, Human! I have some knowledge on this subject, as I am a large language model myself. What specific aspects of integrating external knowledge are you interested in?
Human: I just want to analyze the different possibilities. What can you thif of?
AI:  Well, there are a few different approaches to integrating external knowledge with large language models. One method is to use pre-trained knowledge graphs, which contain structured information about different entities and their relationships. Another approach is to use text corpora, which contain unstructured information and can be used to train the language model on specific domains or topics. Additionally, some models use both structured and unstructured data to enhance their understandin

ConversationBufferMemory는 아래와 같은 장단점이 있다.

|장점|단점|
|------|---|
|모든 정보를 저장을 하고 있기 때문에 LLM에게 최대치의 정보를 제공한다.|사용되는 토큰이 많다는 것은 비용이 높고 응답속도가 느리다는 뜻이다.|
|모든 것을 저장한다는 것은 간단하고 직관적이다.|아주 긴 대화는 모두 기억을 할 수 없다. 왜냐하면 LLM의 토큰 한계치에 걸릴 수 있다. gtp-3.5-turbo는 최대 4096 토큰이 제한이다.|

### ConversationSummaryMemory

<b>ConversationMemory</b>를 사용할 때에는 소비되는 토큰의 양이 급격하게 늘어나면서 LLM이 제한을 하고 있는 context window 크기를 넘어가게 된다는 것을 알 수 있었다. 이렇게 많은 토큰 사용을 줄이기 위해 <b>ConversationSummaryMemory</b>를 사용할 수 있다. 이름이 의미하듯이 이 메모리는 대화 기록(history)를 요약을 해서 {history}에 넘기게 된다.

In [10]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

ConversationSummaryMemory를 초기화할 때의 파라미터로 llm이 들어가는 것을 보면, 요약을 하기 위해 다시 llm을 사용한다는 것을 알 수 있다. 아래 코드로 확인해 보자.

In [11]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


(역주)위 코드와 내용을 음미해 보면 왜 ConversationSummaryMemory가 langchain.chains 패키지에 속하는 지를 알 수 있다. input -> summary prompt -> llm -> summarized input + 현재 input -> llm -> 최종결과가 발생하는 일련의 체인/스텝/파이프라인을 통해서 실행이 되지 않는가?


In [12]:
# count_token없이 우리는 곧바로 conversation_sum("Good morning AI!")라고 호출 할 수 있지만
# 계속해서 토큰 사용량을 체크하기 위해서 기존처럼 사용하자.
count_tokens(
    conversation_sum,
    "Good morning AI!"
)

총 소요 토큰: 281


' Good morning Human! How are you today? I am feeling great as always, ready to assist you with any questions or tasks you may have.'

In [13]:
count_tokens(
    conversation_sum,
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

총 소요 토큰: 453


' Hello there! I hope you are doing well today. I am always happy to assist you in any way I can. Could you provide me with more information about your interest in integrating Large Language Models with external knowledge? I have access to a vast amount of information and would be happy to share it with you.'

In [14]:
count_tokens(
    conversation_sum, 
    "I just want to analyze the different possibilities. What can you think of?"
)

총 소요 토큰: 518


' Hello there! How are you doing today? Is there anything I can help you with? I have access to a vast amount of information and can assist you with integrating Large Language Models with external knowledge. Would you like me to provide you with some specific examples or details to help with your analysis?'

In [15]:
count_tokens(
    conversation_sum, 
    "Which data source types could be used to give context to the model?"
)

총 소요 토큰: 658


' There are a variety of data source types that could be used to provide context to the model. Some examples include text corpora, knowledge graphs, databases, and even social media data. These sources can provide information on a wide range of topics and domains, such as history, science, literature, and current events. Is there a specific topic or domain you are interested in? I can provide specific examples or details to help with your analysis.'

In [16]:
count_tokens(
    conversation_sum, 
    "What is my aim again?"
)

총 소요 토큰: 665


' Your aim is to integrate Large Language Models with external knowledge in order to improve their performance and accuracy. Would you like me to assist you with accessing a vast amount of information for this task?'

위 출력을 살펴보면 요약된 내용도 LLM이 충분히 원래의 목적을 잘 알 수 있도록 내용을 저장하고 있다는 것을 추측할 수 있다. 실제 요약의 내용은 아래와 같이 확인할 수 있다.

In [17]:
print(conversation_sum.memory.buffer)


The human greets the AI and the AI responds in a friendly manner, offering assistance. The human explains their interest in integrating Large Language Models with external knowledge and the AI offers to assist with access to a vast amount of information. The human then asks about data source types that could be used to provide context to the model, to which the AI responds by mentioning a variety of options such as text corpora, knowledge graphs, and social media data. The AI also offers to provide specific examples or details to help with the human's analysis, and asks for a specific topic or domain of interest. The human clarifies their aim of improving the performance and accuracy of Large Language Models by integrating external knowledge, and the AI offers to assist with accessing information.


ConversationBufferMemory와 ConversationSummaryMemory가 사용하는 토큰의 갯수를 그래프로 그려보면 대략적으로(temperature와 사용되는 LLM 모델에 따라서 토큰 갯수는 매번 약간씩 달라진다) 아래와 같다.

![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2Fcf263bd27fc3a9a80cbba68b40ddcdde55d05e29-2788x1427.png&w=3840&q=75)

<b>ConversationSummaryMemory</b>의 장단점은 아래와 같이 정리할 수 있다.

|장점|단점|
|------|---|
|긴 대화의 토큰 개수를 줄일 수 있다.|요약하느라 llm을 또 사용하기 때문에 초기(작은) 대화에는 더 많은 토큰이 사용될 수 있다.|
|더 킨 대화가 가능하다.|대화 기록을 기억하는 능력은 전적으로 요약하는 llm의 능력에 달려있다.|
|그나마 좀 쉬운 구현방법이다. 간단하며 이해가 쉽다.|요약에 LLM을 통해 토큰이 소비된다. 그래서 비용이 증가한다.|

요약을 하기 때문에 토큰 사용량을 줄이기는 하지만 그래도 언젠가는 context window 한계는 넘어가게 되어 있다.

### ConversationBufferWindowMemory
<b>ConversationBufferWindowMemory</b>는 제일 위에서 본 <i>"buffer memory"</i>와 같은 방법으로 동작하지만 메모리에다가 window를 추가한 것이다. window를 추가했다는 뜻은 최근 몇 개의 상호작용(대화)만 저장을 하고 나머지는 잊어버린다는 뜻이다. 아래와 같이 사용할 수 있다.

In [23]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)


<b>k=1</b>이라는 뜻은 최근 하나의 대화(사용자와 AI)만을 기억을 한다는 뜻이다. 아래의 일련의 코드를 통해서 알 수 있다.

In [24]:
count_tokens(
    conversation_bufw, 
    "Good morning AI!"
)

총 소요 토큰: 98


" Good morning to you too, human! It's a beautiful day today, with clear blue skies and a temperature of 75 degrees Fahrenheit. Do you have any plans for the day? "

In [25]:
count_tokens(
    conversation_bufw, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

총 소요 토큰: 236


" That's a fascinating topic! Large Language Models, also known as LLMs, have been gaining a lot of attention lately for their ability to generate human-like text. They work by processing a large amount of text data and learning patterns and structures to generate new text. As for integrating them with external knowledge, there are some approaches being researched, such as using knowledge graphs or external databases to supplement the LLMs' knowledge. However, there are still challenges in effectively integrating these different sources of information. Is there a specific aspect of this topic that you are interested in exploring?"

In [26]:
count_tokens(
    conversation_bufw, 
    "I just want to analyze the different possibilities. What can you think of?"
)

총 소요 토큰: 344


' Well, one possibility could be using LLMs to enhance the accuracy of search engines. By integrating external knowledge, the LLMs could better understand the intent behind a search query and provide more relevant results. Another possibility could be using LLMs to generate personalized content for users by combining their preferences and interests with external knowledge. Additionally, LLMs could also be used for natural language processing tasks, such as chatbots, by incorporating external knowledge to improve the responses and understanding of human language. These are just a few ideas, but there are likely many more possibilities to explore. Is there a specific area or use case you would like me to provide more information on?'

In [27]:
count_tokens(
    conversation_bufw, 
    "Which data source types could be used to give context to the model?"
)

총 소요 토큰: 364


' There are several types of data sources that could be used to provide context to an LLM model. These could include structured data sources, such as databases or spreadsheets, which contain information organized in a specific format. Unstructured data sources, such as text documents or images, could also be used to provide context through natural language processing and computer vision techniques. Additionally, external knowledge bases, such as Wikipedia or Wikidata, could be integrated to provide a broader understanding of concepts and relationships. Social media data and user-generated content could also be leveraged to provide real-time context and insights. Ultimately, the best data sources to use will depend on the specific application and goals of the LLM model.'

In [28]:
count_tokens(
    conversation_bufw, 
    "What is my aim again?"
)

총 소요 토큰: 250


' I am sorry, I do not have enough context to accurately determine your specific aim. Could you please provide more information so I can assist you better?'

기억을 못해서 답을 할 수 없다는 결과가 나왔다. 아래와 같이 메모리에 무엇이 저장되어 있는 지를 알 수 있다.

In [29]:
bufw_history = conversation_bufw.memory.load_memory_variables(
    inputs=[]
)['history']

print(bufw_history)

Human: What is my aim again?
AI:  I am sorry, I do not have enough context to accurately determine your specific aim. Could you please provide more information so I can assist you better?


비록 ConversationBufferWindowMemory가 오래된 과거의 상호작용을 기억을 못하지만, 토큰 사용량을 제어하는 데에는 효과적이다. k=6으로 해서 좀 더 긴 대화를 테스트를 해보면 아래와 같은 그래프를 그릴 수 있다.

![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2Fab6cd8ea8067fecb668cd6a017eff60f7a320d8b-3079x1427.png&w=3840&q=75)

최근 대화만을 가지고 채팅을 하는 것이 우리의 애플리케이션에 적합하다면 ConversationBufferWindowMemory가 적합하다.

### ConversationSummaryBufferMemory
ConversationSummaryMemory와 ConversationBufferWindowMemory를 결합하여 최대 토큰값을 조절하면서 요약을 하는 것도 가능한데 이럴 때에는 이것을 쓰면 된다.

In [30]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

conversation_sum_bufw = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(
        llm=llm,
        max_token_limit=650
    )
)


이것을 사용하면 buffer window에 의해서 제한이 있더라도 과고 대화를 요약을 해서 저장을 할 수 있다. 이 메모리의 장단점은 아래와 같다.

|장점|단점|
|------|---|
|요약이 가능하므로 더 오래된 상호작용을 기억할 수 있다.|요약하느라 llm을 또 사용하기 때문에 초기(작은) 대화에는 더 많은 토큰이 사용될 수 있다.|
|Buffer 덕분에 정보를 잃어버리는 것을 방지할 수 있다.|최근의 대화는 raw(요약없이) 형태로 저장을 한다. 따라서 토큰 갯수를 조금 더 많이 사용한다.|

전체적으로 테스트를 해 본 결과 사용하는 토큰과 기억하는 상호작용의 횟수의 관계는 아래 그래프와 같이 조사되었다.

![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F839a4c11b780250f5ae3adeb1d7873c7002ae21b-3627x1427.png&w=3840&q=75)

### 그 외 메모리 타입
그 외에도 <b>ConversationKnowledgeGraphMemory</b>와 <b>ConversationEntityMemory</b>와 같은 것들이 있다. 